In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
# from sklearn.feature_extraction.text impor
import pickle


In [2]:
data=pd.read_csv('./Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Data drop of unnecessary colums
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Encode categorical varibalel
label_encoder_gender=LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## OneHotEncoder of Geography
onehot_encode_geo=OneHotEncoder()
geo_data = onehot_encode_geo.fit_transform(data[['Geography']])
geo_data


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
onehot_encode_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
print(len(geo_data.toarray()))

10000


In [8]:
geo_data_df= pd.DataFrame(geo_data.toarray(), columns=onehot_encode_geo.get_feature_names_out(['Geography']))
geo_data_df



,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
## concatinate the geo data
data=pd.concat([data.drop(['Geography'], axis=1), geo_data_df], axis=1)

In [10]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

In [11]:
with open('onehot_encode_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encode_geo, file)

In [12]:
X = data.drop(['Exited'], axis=1)
Y = data['Exited']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.25, random_state=64)
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [13]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [15]:
model = Sequential([
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
]
)

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [18]:
log_dir = 'logs/fit' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
callback_of_tenserBoard = TensorBoard(log_dir, histogram_freq=1)

In [19]:
# Stoping
callable_stoping = EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=20)

In [20]:
model.fit(X_train,Y_train,validation_data=[X_test, Y_test], epochs=100, callbacks=[callback_of_tenserBoard, callable_stoping])

Epoch 1/100
79/79 [==============================] - 1s 4ms/step - loss: 0.4609 - accuracy: 0.7948 - val_loss: 0.4098 - val_accuracy: 0.8219
Epoch 2/100
79/79 [==============================] - 0s 2ms/step - loss: 0.3992 - accuracy: 0.8328 - val_loss: 0.3630 - val_accuracy: 0.8541
Epoch 3/100
79/79 [==============================] - 0s 2ms/step - loss: 0.3718 - accuracy: 0.8432 - val_loss: 0.3633 - val_accuracy: 0.8437
Epoch 4/100
79/79 [==============================] - 0s 2ms/step - loss: 0.3579 - accuracy: 0.8496 - val_loss: 0.3536 - val_accuracy: 0.8599
Epoch 5/100
79/79 [==============================] - 0s 2ms/step - loss: 0.3586 - accuracy: 0.8540 - val_loss: 0.3681 - val_accuracy: 0.8499
Epoch 6/100
79/79 [==============================] - 0s 2ms/step - loss: 0.3469 - accuracy: 0.8564 - val_loss: 0.3581 - val_accuracy: 0.8520
Epoch 7/100
79/79 [==============================] - 0s 2ms/step - loss: 0.3340 - accuracy: 0.8612 - val_loss: 0.3606 - val_accuracy: 0.8520
Epoch 8/100
7

In [21]:
model.save('mode.h5')

/Users/sidrastogi/Dev2/GenAI/Ann Classification/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
%load_ext tensorboard

In [23]:
%tensorboard --logdir logs/fit20250511-141926/

Reusing TensorBoard on port 6007 (pid 5082), started 0:04:34 ago. (Use '!kill 5082' to kill it.)

In [96]:
text_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2, 
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [97]:
from tensorflow.keras.models import load_model

In [98]:
model = load_model('./mode.h5')

In [99]:
with open('./label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

In [100]:
with open('./onehot_encode_geo.pkl', 'rb') as file:
    onehot_encode_geo = pickle.load(file)

In [101]:
with open('./scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [102]:
text_data = pd.DataFrame([text_data])

In [ ]:
text_data['Gender'] = label_encoder_gender.transform(text_data['Gender'])


In [105]:
geo_encoded = onehot_encode_geo.transform([text_data['Geography']]).toarray()

/Users/sidrastogi/Dev2/GenAI/Ann Classification/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [106]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encode_geo.get_feature_names_out(['Geography']))

In [107]:
text_data = pd.concat([text_data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [108]:
text_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [109]:
input_scaled = scaler.transform(text_data)
input_scaled

array([[-0.53998226,  0.90088523,  0.08938627, -0.68058838, -0.28106287,
         0.80099415,  0.66527071,  0.95923036, -0.87620419,  1.00080032,
        -0.57365519, -0.58166121]])

In [110]:
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 35ms/step


array([[0.01838904]], dtype=float32)

In [111]:
prediction_pro = prediction[0][0]

In [112]:
prediction_pro

0.018389042